In [30]:
import cvxpy as cp
import numpy as np
import networkx as nx


#----------------- Butterfly flow=1.5 --------------------------------
inputnodes = [0,6]
inputedges = [(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)]

C = {
    (0,5):1,
    (0,3):1,
    (3,4):1,
    (4,5):1,
    (4,6):1,
    (5,2):1,
    (3,1):1,
    (1,6):1,
    (6,2):1
}

non_member = [3,4,5,6]
source = 0
receivers = [1,2]

#----------------- C(3,2), flow=1.5/1.8--------------------------------

# inputnodes = [0,6]
# inputedges = [(0,4),(0,5),(0,6),(4,1),(4,2),(5,1),(5,3),(6,2),(6,3)]

# C = {
#     (0,4):1,
#     (0,5):1,
#     (0,6):1,
#     (4,1):1,
#     (4,2):1,
#     (5,1):1,
#     (5,3):1,
#     (6,2):1,
#     (6,3):1,
# }

# non_member = [4,5,6]
# source = 0
# receivers = [1,2,3]



#----------------- Fig7. in mflow paper, flow=13.5/13.5 --------------------------------

# inputnodes = [0,7]
# inputedges = [(1, 3), (1, 4), (3, 4), (3, 5), (4, 5), (2, 3), (2, 5), (0, 5), (0, 7), (2, 7), (6, 7), (2, 6), (1, 6), (0, 4), (4, 7), (4, 6)]

# C = {(1,3):4, (1,4):8, (3,4):1, (3,5):1, (4,5):1, (2,3):4, (2,5):4, (0,5):4, (0,7):4, (2,7):4, (6,7):1, (2,6):4, (1,6):4, (0,4):8, (4,7):1, (4,6):1}

# non_member = [3,4,5,6,7]
# source=0
# receivers = [1,2]


#----------------- C(4,3), flow=2/2.667 --------------------------------

# inputnodes = [0,7]
# inputedges = [(0,5),    (0,6),   (0,7),    (0,8),    (5,1),    (5,2),    (5,3),    (6,1),    (6,2),    (6,4),    (7,1),    (7,3),   (7,4),   (8,2),    (8,3),    (8,4)]

# C = {
#     (0,5):1,
#     (0,6):1,
#     (0,7):1,
#     (0,8):1,
#     (5,1):1,
#     (5,2):1,
#     (5,3):1,
#     (6,1):1,
#     (6,2):1,
#     (6,4):1,
#     (7,1):1,
#     (7,3):1,
#     (7,4):1,
#     (8,2):1,
#     (8,3):1,
#     (8,4):1
# }

# non_member = [5,6,7,8]
# source = 0
# receivers = [1,2,3,4]

#----------------- C(4,2), flow=1.333/1.778 --------------------------------
# inputnodes = [0,10]
# inputedges = [(0,7),(0,8),(0,9),(0,10),(7,1),(7,2),(7,3),(8,1),(8,5),(8,4),(9,2),(9,4),(9,6),(10,3),(10,5),(10,6)]
# C = {(0, 7): 1, (0, 8): 1, (0, 9): 1, (0, 10): 1, (7, 1): 1, (7, 2): 1, (7, 3): 1, (8, 1): 1, (8, 5): 1, (8, 4): 1, (9, 2): 1, (9, 4): 1, (9, 6): 1, (10, 3): 1, (10, 5): 1, (10, 6): 1}

# non_member = [7,8,9,10]
# source=0
# receivers = [1,2,3,4,5,6]


#----------------- C(5,4), flow=2.5/3.571 --------------------------------
# inputnodes = [0,10]
# inputedges = [(0,6),(0,7),(0,8),(0,9),(0,10),(6,1),(6,2),(6,3),(6,4),(7,1),(7,2),(7,3),(7,5),(8,1),(8,2),(8,4),(8,5),(9,1),(9,3),(9,4),(9,5),(10,2),(10,3),(10,4),(10,5)]
# C = {(0, 6): 1, (0, 7): 1, (0, 8): 1, (0, 9): 1, (0, 10): 1, (6, 1): 1, (6, 2): 1, (6, 3): 1, (6, 4): 1, (7, 1): 1, (7, 2): 1, (7, 3): 1, (7, 5): 1, (8, 1): 1, (8, 2): 1, (8, 4): 1, (8, 5): 1, (9, 1): 1, (9, 3): 1, (9, 4): 1, (9, 5): 1, (10, 2): 1, (10, 3): 1, (10, 4): 1, (10, 5): 1}
# non_member = [6,7,8,9,10]
# source=0
# receivers = [1,2,3,4,5]

#----------------- C(5,2), flow=1.25/1.786 --------------------------------

# inputnodes = [0, 10]
# inputedges = [(0,1), (0,2), (0,3), (0,4), (0,5), (1,6), (2,6), (1,7), (3,7), (1,8), (4,8), (1,9), (5,9), (2,10), (3,10), (2,11), (4,11), (2,12), (5,12), (3,13), (4,13), (3,14), (5,14), (4,15), (5,15)]
# C = {(0, 1): 1, (0, 2): 1, (0, 3): 1, (0, 4): 1, (0, 5): 1, (1, 6): 1, (2, 6): 1, (1, 7): 1, (3, 7): 1, (1, 8): 1, (4, 8): 1, (1, 9): 1, (5, 9): 1, (2, 10): 1, (3, 10): 1, (2, 11): 1, (4, 11): 1, (2, 12): 1, (5, 12): 1, (3, 13): 1, (4, 13): 1, (3, 14): 1, (5, 14): 1, (4, 15): 1, (5, 15): 1}
# non_member = [1, 2, 3, 4, 5]
# source = 0
# receivers = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15]




#----------------- coding started  --------------------------------
preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)



def find_all_paths(G, source, receivers):
    all_paths = {}
    for receiver in receivers:
        paths = list(nx.all_simple_paths(G, source=source, target=receiver))
        all_paths[receiver] = paths
        print(f"All paths from {source} to {receiver}: {paths}")
    return all_paths

all_paths = find_all_paths(G, source, receivers)
print(all_paths)




#----------------------------- Cflow LP --------------------------#


def cflow(source, receivers, non_member, undirected_edges, edges, C):
    f_star = cp.Variable()  # Target flow rate


    orientation_constraints = []
    flow_constraints = []
    equalrate_constraints = []

    c =  {a : cp.Variable() for a in edges}

    for a in edges:
        orientation_constraints += [c[a] >= 0]      #for every in 2E

    for a1 in undirected_edges:         # for every in E
        a2 = (a1[1], a1[0]) 
        orientation_constraints += [ c[a1] + c[a2] == C[a1]]


    fia = {(i, a): cp.Variable() for a in edges for i in receivers}

    for i in receivers: 
        for a in edges:
            flow_constraints += [fia[i,a] <= c[a]]
            flow_constraints += [fia[i,a] >= 0]


    for i in receivers:
        for j in non_member:
            incoming_edges = [(u, v) for u, v in edges if v == int(j)] 
            outcoming_edges = [(u, v) for u, v in edges if u == int(j)] 
            incoming_flow = sum(fia[i, x] for x in incoming_edges)
            outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
            flow_constraints += [incoming_flow == outcoming_flow]

        incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  
        
        outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i)) 
        incoming_flow_reciever = sum(fia[i, x] for x in edges if x[1] == int(i))  

        flow_constraints += [incoming_flow_source == 0]
        flow_constraints += [outcoming_flow_reciever == 0]
        equalrate_constraints += [f_star == incoming_flow_reciever]


    objective = cp.Maximize(f_star)    
    constraints = orientation_constraints + flow_constraints + equalrate_constraints
    problem = cp.Problem(objective, constraints)


    problem.solve()

    print("Optimal Target Flow Rate (f*):", f_star.value)

    optimal_flow = f_star.value
    flowia = {(i, a): round(float(fia[i,a].value), 6) for a in edges for i in receivers}

    return(optimal_flow, flowia)


optimal_flow, flowia = cflow(source, receivers, non_member, undirected_edges, edges, C)
print(flowia)

#-----------------------------  Splitting cflow Function with LP ---------------------------#

# single matrix for one receiver
def create_edge_path_matrix(G, all_paths, receiver):
    edges = list(G.edges())
    paths = all_paths[receiver]

    matrix = np.zeros((len(edges), len(paths)), dtype=int)
    edge_index = {edge: idx for idx, edge in enumerate(edges)}
    
    for path_idx, path in enumerate(paths):
        for u, v in zip(path[:-1], path[1:]):  # Create edges from path
            if (u, v) in edge_index:
                matrix[edge_index[(u, v)], path_idx] = 1
    
    return matrix

# save matrices for all receivers
matrices = {}
for receiver in receivers:
    matrices[receiver] = create_edge_path_matrix(G, all_paths, receiver)
    print(f"Matrix for receiver {receiver}:\n{matrices[receiver]}\n")



def optimize_path_flows(matrix, edge_flows):
    num_paths = matrix.shape[1]         # column number
    print("number of paths is", num_paths)
    num_edges = matrix.shape[0]         # row number
    x = cp.Variable(num_paths, nonneg=True)

    objective = cp.Maximize(cp.sum(x))
    constraints = [matrix @ x <= edge_flows]

    problem = cp.Problem(objective, constraints)
    result = problem.solve()
    return x.value, result

# Applying the optimization to each receiver
optimized_flows = {}
for receiver, matrix in matrices.items():
    # Get the flows for each edge for this receiver from flowia
    edge_flows = np.array([flowia.get((receiver, edge), 0) for edge in G.edges()])

    # Run the optimization
    optimized_path_flow, max_flow_sum = optimize_path_flows(matrix, edge_flows)
    optimized_flows[receiver] = (optimized_path_flow, max_flow_sum)
    print(f"Optimized path flows for receiver {receiver}: {optimized_path_flow}")
    print(f"Maximum sum of flows for receiver {receiver}: {max_flow_sum}\n")


significant_paths = {}

threshold = 1e-6

for receiver, (optimized_path_flow, _) in optimized_flows.items():
    # Fetch the paths from all_paths for this receiver
    paths = all_paths[receiver]
    
    # Filter paths based on the optimized path flow values
    non_zero_paths = [paths[i] for i in range(len(paths)) if optimized_path_flow[i] > threshold]
    
    # Save or print the non-zero paths
    significant_paths[receiver] = non_zero_paths
    print(f"Significant paths for receiver {receiver} with non-zero flow values:")
    for path in non_zero_paths:
        print(path)

print(significant_paths)


#-----------------------------  combine to steiner tree ---------------------------#

from itertools import product

# Generate all possible combinations of paths, one from each receiver's significant paths
path_combinations = list(product(*[significant_paths[receiver] for receiver in significant_paths]))

resulting_graphs = []

# Create a graph for each combination of paths
for combination in path_combinations:
    # Create a new directed graph for this combination
    combo_graph = nx.DiGraph()
    
    # Add paths to the graph
    for path in combination:
        nx.add_path(combo_graph, path)
    
    # Add this graph to the list of resulting graphs
    resulting_graphs.append(combo_graph)

print(f"Total combinations (resulting graphs) created: {len(resulting_graphs)}")
edge_lists = []

for graph in resulting_graphs:
    edge_list = list(graph.edges())
    edge_lists.append(edge_list)

print("resulting steiner trees are ", edge_lists)



#-----------------------------  packing steiner trees LP ---------------------------#


n_graphs = len(edge_lists)              # Number of resulting graphs
packing_x = cp.Variable(n_graphs, nonneg=True)
packing_objective = cp.Maximize(cp.sum(packing_x))
packing_constraints = []

for (u, v), capacity in C.items():
    total_flow = cp.sum([packing_x[i] for i, edges in enumerate(edge_lists) if (u, v) in edges or (v, u) in edges])
    
    packing_constraints.append(total_flow <= capacity)

packing_problem = cp.Problem(packing_objective, packing_constraints)
packing_result = packing_problem.solve()
optimized_weights = packing_x.value

print("Optimized weights for each graph:", optimized_weights)
print("Maximum sum of weights:", packing_result)


[ 0  1  2  3  4  5  6  7  8  9 10]
{(4, 0), (3, 7), (3, 10), (4, 15), (3, 13), (0, 2), (9, 5), (11, 2), (0, 5), (10, 3), (1, 0), (1, 6), (2, 11), (1, 9), (15, 5), (6, 2), (7, 1), (3, 0), (14, 3), (5, 0), (4, 8), (5, 9), (4, 11), (9, 1), (5, 12), (10, 2), (11, 4), (0, 1), (5, 15), (13, 4), (0, 4), (15, 4), (6, 1), (2, 10), (1, 8), (7, 3), (12, 2), (12, 5), (14, 5), (3, 14), (8, 4), (4, 13), (8, 1), (5, 14), (0, 3), (2, 0), (13, 3), (1, 7), (2, 6), (2, 12)}
All paths from 0 to 6: [[0, 1, 6], [0, 1, 7, 3, 10, 2, 6], [0, 1, 7, 3, 13, 4, 11, 2, 6], [0, 1, 7, 3, 13, 4, 15, 5, 12, 2, 6], [0, 1, 7, 3, 14, 5, 12, 2, 6], [0, 1, 7, 3, 14, 5, 15, 4, 11, 2, 6], [0, 1, 8, 4, 11, 2, 6], [0, 1, 8, 4, 13, 3, 10, 2, 6], [0, 1, 8, 4, 13, 3, 14, 5, 12, 2, 6], [0, 1, 8, 4, 15, 5, 12, 2, 6], [0, 1, 8, 4, 15, 5, 14, 3, 10, 2, 6], [0, 1, 9, 5, 12, 2, 6], [0, 1, 9, 5, 14, 3, 10, 2, 6], [0, 1, 9, 5, 14, 3, 13, 4, 11, 2, 6], [0, 1, 9, 5, 15, 4, 11, 2, 6], [0, 1, 9, 5, 15, 4, 13, 3, 10, 2, 6], [0, 2, 6], [0, 2, 1